In [16]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation
import re
from nltk.stem import SnowballStemmer
from autocorrect import Speller
from collections import Counter 
import timeit
import spacy
from langdetect import detect

In [17]:
contractions_dict = {     
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I had",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "iit will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that had",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there had",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they had",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [18]:
stopword = stopwords.words('english')
wordnet_lemmatizer = WordNetLemmatizer()
snowball_stemmer = SnowballStemmer('english')
punct=punctuation+'’'+'“'+'”'+'–'

def expand_contractions(text, contractions_dict):
    """
    expand english contractions
    """
    contractions_pattern = re.compile('({})'.format('|'.join(contractions_dict.keys())),
                                      flags=re.IGNORECASE | re.DOTALL)

    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contractions_dict.get(match) \
            if contractions_dict.get(match) \
            else contractions_dict.get(match.lower())
        expanded_contraction = expanded_contraction
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

def autospell(text):
    """
    correct the spelling of the word.
    """
    spell = Speller(lang='en',fast=True)
    spells = [spell(w) for w in (nltk.word_tokenize(text))]
    return " ".join(spells)



'''def nlp_clean(text):
    text = expand_contractions(text,contractions_dict)
    text = autospell(text)
    text = ' '.join([w.lower().strip('intro').strip('chorus') for w in nltk.word_tokenize(text)])
    text = re.sub('<[^<]+?>','', text)
    text = ''.join(c for c in text if not c.isdigit())
    text = ''.join(c for c in text if c not in punct)
    word_tokens = nltk.word_tokenize(text)
    word_tokens = [wordnet_lemmatizer.lemmatize(word) for word in word_tokens]
    word_tokens = [snowball_stemmer.stem(word) for word in word_tokens]
    result = [word for word in word_tokens if word not in stopword]
    
    return result

'''

"def nlp_clean(text):\n    text = expand_contractions(text,contractions_dict)\n    text = autospell(text)\n    text = ' '.join([w.lower().strip('intro').strip('chorus') for w in nltk.word_tokenize(text)])\n    text = re.sub('<[^<]+?>','', text)\n    text = ''.join(c for c in text if not c.isdigit())\n    text = ''.join(c for c in text if c not in punct)\n    word_tokens = nltk.word_tokenize(text)\n    word_tokens = [wordnet_lemmatizer.lemmatize(word) for word in word_tokens]\n    word_tokens = [snowball_stemmer.stem(word) for word in word_tokens]\n    result = [word for word in word_tokens if word not in stopword]\n    \n    return result\n\n"

In [19]:
df=pd.read_csv('../data/Hot100Data.csv')
df.dropna(inplace=True)
df['lang']=df.Lyrics.apply(detect)
df=df[df['lang']=='en']
df.drop(columns=['Unnamed: 0','lang'], inplace=True)

In [22]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'tagger', 'ner']) #load small english core library
stops = stopwords.words("english") #load stopwords



def normalize(text, lowercase, remove_stopwords):
    text = expand_contractions(text,contractions_dict) #expand english contractions
    text = autospell(text) #correct spelling
    #text = ' '.join([w.lower() for w in nltk.word_tokenize(text)])  #lowercase
    text = re.sub('<[^<]+?>','', text)  #remove brackets
    text = ''.join(c for c in text if not c.isdigit())  #remove numbers
    text = ''.join(c for c in text if c not in punct)  #remove punctuations
    if lowercase:
        text = text.lower()  #lowercase
    text = nlp(text)
    lemmatized = list()
    for word in text:
        lemma = word.lemma_.strip() #tokenize
        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in stops):  #remove stopwords
                lemmatized.append(lemma)
    return lemmatized


df['After_Clean'] = df['Lyrics'].apply(normalize, lowercase=True, remove_stopwords=True)


/Users/jialinshan/anaconda3/lib/python3.7/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [23]:
df.head()

,Year,Performer,Song,Lyrics,After_Clean
0,2019,Post Malone Featuring DaBaby,Enemies,\n\n[Chorus: Post Malone]\nUsed to have friend...,"[chorus, post, malone, use, friend, get, enemy..."
1,2019,"Yella Beezy, Gucci Mane & Quavo",Bacc At It Again,"\n\n[Intro]\nCook that shit up, Quay\n\n[Choru...","[intro, cook, shit, quay, chorus, della, beery..."
2,2019,DaBaby,VIBEZ,"\n\n[Intro]\nLet's go (Yeah, yeah, Neeko, you ...","[intro, let, us, go, yes, yes, neko, make, mot..."
3,2019,NF,When I Grow Up,"\n\n[Verse 1]\nYeah, when I grow up, you know ...","[verse, yes, grow, know, wan, na, take, seat, ..."
4,2019,Post Malone,Hollywood's Bleeding,"\n\n[Intro]\nHollywood's bleeding, vampires fe...","[intro, hollywood, bleed, vampire, feed, darkn..."


In [24]:
df.to_csv('cleandata.csv')

In [60]:
word_count=df.After_Clean.apply(Counter).sum()
result={}
for w in sorted(word_count, key=word_count.get, reverse=True):
        result[w]=word_count[w]

pd.DataFrame(list(result.items()),columns = ['words','counts']).to_csv('count2010.csv')

In [ ]:
for token in about_doc:
     print (token, token.tag_, token.pos_, spacy.explain(token.tag_))